# COVID-19 Data Analysis

假設今天你任職於一個國外的非政府組織，從事資料分析的工作。你的工作就是：即時為部門主管提供最新的新冠疫情圖表，作為高層開會決策時的依據。

其中一個有用的資料來源是：[Coronavirus Source Data](https://ourworldindata.org/coronavirus-source-data) ([下載頁面](https://github.com/owid/covid-19-data/tree/master/public/data))。請使用`owid-covid-data-2021-10-10.csv`這個檔案 (下載於2021/10/10 ，剛好是中華民國國慶)

以下步驟，將會引導你做出全世界每個國家每百萬人的新增個案圖。

首先是讀取資料

In [ ]:
import pandas as pd
df = pd.read_csv('owid-covid-data-2021-10-10.csv')
df

你可以看到總共有65個欄位 (columns)

In [ ]:
i = 0
for column in df.columns:
    i += 1
    print(i, column)

其中以下三個 columns 是我們接下來會用到的資訊：
- `location`: 地理位置，也就是國家
- `date`: 日期
- `new_cases_smoothed_per_million`: 每日新增個案，為七日平均之平滑數字

### Question 1
主管首先要你提供全世界防疫成功的國家案例，你首先想到的就是故鄉台灣，因此你要先篩選 (filter) 出台灣的資料表格。

你思考了一下，知道第一步是要創造出一個 boolean (True/False) 的 Pandas Series。True 代表是台灣的 row，False 代表是其他國家。請問要怎麼做？

In [ ]:
# ANSWER 1 (1 point)

# --- 你的程式從此開始 ---

is_taiwan = df['location'] == 'Taiwan'

# --- 你的程式在此結束 ---

is_taiwan

你可以看到台灣總共有 633 個資料點。在電腦裡 True = 1, False = 0，所以加總起來就是所有 True 的數量。

In [ ]:
is_taiwan.sum()

### Question 2
有了 `is_taiwan` 這個 boolean series 之後，你就可以把所有台灣的資料篩出來。請問要怎麼做？

In [ ]:
# ANSWER 2 (1 point)

# --- 你的程式從此開始 ---

taiwan_df = df.loc[is_taiwan]

# --- 你的程式在此結束 ---

taiwan_df.reset_index(inplace=True, drop=True)
taiwan_df

你可以看到總共有 65 個欄位，而大部分的欄位資料都是空缺的 (`NaN`)。

事實上，**每日新增個案數**是疫情直接的指標。但每個國家的人口數不同，因此我們會以「每百萬人」為衡量單位。此外，有許多人為因素 (例如週末休假) 會導致資料蒐集的波動，因此我們會計算「七日平均之平滑線」。

這就是我們要專注分析的量化指標：`new_cases_smoothed_per_million`

### Question 3
請取出台灣所有 `new_cases_smoothed_per_million` 的資料。結果會是一個 `Series`。

In [ ]:
# ANSWER 3 (1 point)

# --- 你的程式從此開始 ---

taiwan_new_cases = taiwan_df['new_cases_smoothed_per_million']

# --- 你的程式在此結束 ---

taiwan_new_cases

### Question 4
你注意到一開始的那幾天，是沒有資料的 (`NaN`)，因此你想看看總共有幾個可用、不是空白的資料點。

你的結果 `number_of_valid_data` 會是一個整數 (integer)。

提示：你可以使用`dropna()`或是`notna()`

In [ ]:
# ANSWER 4 (1 point)

# --- 你的程式從此開始 ---

number_of_valid_data = len(taiwan_new_cases.notna())

# --- 你的程式在此結束 ---

number_of_valid_data

### Question 5
因為你是要觀察每日新增個案，因此需要取出時間的資料，也就是 `date` 欄位。

In [ ]:
# ANSWER 5 (1 point)

# --- 你的程式從此開始 ---

taiwan_dates = taiwan_df['date']

# --- 你的程式在此結束 ---

taiwan_dates

### Question 6
台灣最高的每日新增個案數是多少？請注意我們使用的數據是**每百萬人**的新增個案，因此你需要依照台灣人口數換算成實際新增個案數。

提示：`.max()`

In [ ]:
# ANSWER 6 (2 point)

taiwan_population = 23570000  # 兩千三百萬人

# --- 你的程式從此開始 ---

max_cases_per_million = taiwan_new_cases.max()

max_cases = int(max_cases_per_million * taiwan_population / 1000000)

# --- 你的程式在此結束 ---

max_cases

回想當時情勢嚴峻，這個數字應該跟你記憶中的數字應該很接近吧？

### Question 7
請問台灣的最高個案數，是何年何月何日發生的？你的結果會是一個字串，看起來像 `'2020-xx-xx'`

提示：`day_of_max_cases` 是一個 `Series` (只有一個日期), 而 `iloc[0]` 是為了要取出那個日期 

In [ ]:
# ANSWER 7 (1.5 point)

# --- 你的程式從此開始 ---

is_max = taiwan_new_cases == taiwan_new_cases.max()

day_of_max_cases = taiwan_dates[is_max]

# --- 你的程式在此結束 ---

day_of_max_cases.iloc[0]

你還記得那天嗎？

資料分析到這步，你發現你已經能作圖呈現給主管，讓他 (她) 看看台灣這個防疫成功的案例。

以下的程式定義了一個函數 `plot()`，供你做出日期折線圖。

這個函數有三個 parameters：
- `label`: 標記 (`str`)
- `x_dates`: Χ 軸日期 (`pd.Series`)
- `y_values`: Y 軸數值 (`pd.Series`)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 7.5]

def plot(label, x_dates, y_values):
    
    global plt  # for exec() to access 'plt' as global variable
    global mdates
    global datetime
    
    # 設定圖表格式
    ax = plt.gca()
    formatter = mdates.DateFormatter("%Y-%m-%d")
    ax.xaxis.set_major_formatter(formatter)
    locator = mdates.MonthLocator()
    ax.xaxis.set_major_locator(locator)
    plt.xticks(rotation=90)

    # 設定 X 軸日期格式
    x_dates = [datetime.datetime.strptime(d, "%Y-%m-%d").date() for d in x_dates]

    # 作圖
    plt.plot(x_dates, y_values, label=label)
    ax.legend(loc="upper left")
    
    return True

### Question 8
要做出日期折線圖，你的 X 軸是 `taiwan_dates`，Y 軸是 `taiwan_new_cases`

In [ ]:
# ANSWER 8 (1.5 point)

# --- 你的程式從此開始 ---

success = plot(
    label='Taiwan',
    x_dates=taiwan_dates,
    y_values=taiwan_new_cases,
)

plt.show()

# --- 你的程式在此結束 ---

success

結果麻煩的事來了！你的主管超沒概念！He/She had no sense!

他 (她) 說台灣五月底那個 peak (峰值) 看起來很嚴重，其實也不能當作成功案例吧？

你的主管沒有注意到的是，Y 軸的最高值不過是每百萬人 25 人左右。

於是，你決定把其他國家的數據也加進來，徹底說服你的主管 (你可以自由地加進你想要看的國家)。

In [ ]:
labels = [
    'United States', 
    'United Kingdom',
    'Germany',
    'Iceland',
    'Japan',
    'Korea',
    'Vietnam',
    'Taiwan',
]

for label in labels:
    is_location = df['location'] == label
    columns = ['date', 'new_cases_smoothed_per_million']

    location_df = df.loc[is_location, columns]
    
    plot(
        label=label,
        x_dates=location_df['date'],
        y_values=location_df['new_cases_smoothed_per_million'],
    )

plt.show()

你看到台灣的曲線了嗎？